# TP03 : Naive Bayes


## I. Implémentation

Pour estimer la vraisemblance, il y a plusieurs modèles (lois):
- Loi multinomiale : pour les caracétristiques nominales
- Loi de Bernoulli : lorsqu'on est interressé par l'apparence d'une caractéristique ou non (binaire)
- loi normale : pour les caractéristiques numériques

Dans ce TP, on va implémenter Naive Bayes pour les caractéristiques nominales (loi multinomiale). 
Dans notre modèle, on veut stocker des statistiques et pas des probabilités. 
L'intérêt est pour faciliter la mise à jours des statistiques (si par exemple, nous avons un autre dataset et on veut enrichir le modèle ; donc,  ce n'ai pas la peine d'entraîner de nouveau sur l'ancien dataset)

Ici, on va utiliser le dataset "jouer" contenant des caractéristiques nominales.

In [ ]:
import numpy as np
import pandas as pd 


jouer = pd.read_csv("datasets/jouer.csv")

X_jouer = jouer.iloc[:, :-1].values # Premières colonnes 
Y_jouer = jouer.iloc[:,-1].values # Dernière colonne 

# Afficher le dataset "jouer"
jouer

### I.1. Entraînement de la probabilité antérieure
Etant donné le vecteur de sortie $Y$, on doit calculer la probabilité de chaque classe (différentes valeurs de $Y$)

$$p(c_k) = \frac{|\{y / y \in Y \text{ et } y = c_k\}|}{|Y|}$$

Etant donné un vecteur $Y$ contenant des échantillons sous forme des catégories nominales (les classes dans ce cas), la fonction doit récupérer des statistiques afin de pouvoir calculer la probabilité antérieure de chaque classe. Donc, elle doit retourner  :
- Un vecteur contenant les noms des classes
- Un vecteur contenant les nombres d'occurrences de chaque classe dans le premier vecteur

In [ ]:
# TODO Compléter la fonction des stastistiques pour la probabilité antérieure
def stat_anterieure(Y): 
    cls = np.unique(Y)
    freq = []
    # compléter à partir d'ici
    return cls, np.array(freq)

#=====================================================================
# TEST UNITAIRE
#=====================================================================
# Resultat : (array(['non', 'oui'], dtype=object), array([5, 9]))
#---------------------------------------------------------------------

stat_anterieure(Y_jouer)

### I.2. Entraînement de la probabilité de vraissemblance (loi multinomiale)

Notre modèle doit garder le nombre des différentes valeurs d'une caractéristique $A$ et le nombre de ces valeurs dans chaque classe.
Donc, étant donné un vecteur d'une caractéristique $A= X[:,j]$, un autre des $Y$ et un $C$ contenant la liste des classes, la fonction d'entrainement doit retourner : 
- $V$ : un vecteur contenant les différenntes catégories de $A$
- une matrice conntenant le nombre d'occurrences de chaque catégorie de $V$ dans chaque classe  : 
   - les lignes représentent les catégories $v \in V$ de la caréctéristique $A$
   - les colonnes représentent les classes $c \in C$ de $Y$


In [ ]:
# TODO Compléter la fonction des statistiques de vraissemblance (1 seule caractéristique)
def stat_vraissemblance_1(A, Y, C): 
    freq = []
    V = np.unique(A)
    # compléter à partir d'ici
    return V, np.array(freq)

#=====================================================================
# TEST UNITAIRE
#=====================================================================
# Resultat : (array(['ensoleile', 'nuageux', 'pluvieux'], dtype=object), array([[3, 2],
#         [0, 4],
#         [2, 3]]))
#---------------------------------------------------------------------
C_t = np.array(["non", "oui"])
stat_vraissemblance_1(X_jouer[:, 0], Y_jouer, C_t)

### I.3. Entraînement loi multinomiale

Notre modèle ($\theta_{X, C}$) doit garder des statistiques sur les classes et aussi sur chaque catégorie de chaque caractéristique. Pour ce faire, on va représenter $\theta$ comme un vecteur : 
- $\theta[N+1]$ est un vecteur de $N$ éléments représentants des statistiques sur chaque caractéristique $j$, plus un élément (le dernier) pour les statistiques sur les classes.
- Chaque élément est un dictionnaire (HashMap en Java)
- Un élément des caractéristiques contient deux clés : 
    - **val** : pour récupérer la liste des noms des catégories de la caractéristique
    - **freq**: pour récupérer une matrice représentant la fréquence de chaque caractéristique dans chaque classe
- Un élément des classes contient deux clés : 
    - **cls** : pour récupérer la liste des noms des classes
    - **freq**: pour récupérer la liste des fréquences de chaque classe

In [ ]:
# La fonction qui entraine Théta sur plusieurs caractéristiques
# Rien à programmer ici
# Notre théta est une liste des dictionnaires;
# chaque dictionnaire contient la liste des catégories et la matrice des fréquences dela caractéristique respective à la colonne de X
# On ajoute les statistiques antérieures des classes à la fin de résultat
def entrainer_multi(X, Y): 
    Theta = []
    
    stats_c = {}
    stats_c["cls"], stats_c["freq"] =  stat_anterieure(Y)
    
    for j in range(X.shape[1]): 
        stats = {}
        stats["val"], stats["freq"] =  stat_vraissemblance_1(X[:, j], Y, stats_c["cls"])
        Theta.append(stats)
    
    Theta.append(stats_c)
    return Theta


#=====================================================================
# TEST UNITAIRE
#=====================================================================
# Resultat : 
#   [{'freq': array([[3, 2],
#          [0, 4],
#          [2, 3]]),
#   'val': array(['ensoleile', 'nuageux', 'pluvieux'], dtype=object)},
#  {'freq': array([[2, 2],
#          [2, 4],
#          [1, 3]]), 'val': array(['chaude', 'douce', 'fraiche'], dtype=object)},
#  {'freq': array([[4, 3],
#          [1, 6]]), 'val': array(['haute', 'normale'], dtype=object)},
#  {'freq': array([[2, 6],
#          [3, 3]]), 'val': array(['non', 'oui'], dtype=object)},
#  {'cls': array(['non', 'oui'], dtype=object), 'freq': array([5, 9])}]
#---------------------------------------------------------------------
Theta_jouer = entrainer_multi(X_jouer, Y_jouer)

Theta_jouer

### I.4. Estimation de la probabilité de vraissemblance (loi multinomiale)
L'équation pour estimer la vraisemblance 
$$ P(X_j=v|y=c_k) = \frac{|\{ y \in Y / y = c_k \text{ et } X_j = v\}|}{|\{y = c_k\}|}$$

Si, dans le dataset de test, on veut calculer la probabilité d'une valeur $v$ qui n'existe pas dans le dataset d'entrainnement ou qui n'existe pas pour une classe donnée, on aura une probabilité nulle. Ici, on doit appliquer une fonction de lissage qui donne une petite probabilité aux données non vues dans l'entrainnement. Le lissage qu'on va utiliser est celui de Lidstone. Lorsque $\alpha = 1$ on l'appelle lissage de Laplace.
$$ P(X_j=v|y=c_k) = \frac{|\{ y \in Y / y = c_k \text{ et } X_j = v\}| + \alpha}{|\{y = c_k\}| + \alpha * |V|}$$
Où: 
- $\alpha$ est une valeur donnée 
- $V$ est l'ensemble des différentes valeurs de $f_j$ (le vocabulaire)

Etant donné : 
- $\theta_j$ les paramètres de la caractéristique $j$ représentées comme dictionnaire
    - **val** : pour récupérer la liste des noms des catégories de la caractéristique (vocabulaire $V$)
    - **freq**: pour récupérer une matrice représentant la fréquence de chaque caractéristique dans chaque classe. C'est une matrice $|V|\times|C|$
- $v$ la valeur de la caractéristique $j$ dont on veut utiliser pour calculer les probabilités
- $\theta_c$ les paramètres des classes $C$ représentées comme dictionnaire
    - **cls** : pour récupérer la liste des noms des classes
    - **freq**: pour récupérer la liste des fréquences des classes
    
Cette fonction doit retourner : 
- Une liste $P[|C|]$ contenant les probabilités de la catégorie $v$ de $X_j$ sur toutes les classes $C$ 
- Elle doit prendre en cosidération le cas où la valeur $v$ n'existe pas dans le modèle entraîné

In [ ]:
# TODO compléter la fonction qui calcule la vraissamblance d'une valeur donnée
def P_vraiss_multi(Theta_j, v, Theta_c, alpha=0.): 
    ind = np.where(Theta_j["val"] == v)[0] #une liste des indices où se trouve la valeur v dans Theta_j["val"]
    # si la liste est vide, la valeur n'existe pas dans théta, sinon l'indice est dans ind[0]
    # compléter à partir d'ici
    return None

#=====================================================================
# TEST UNITAIRE
#=====================================================================
# Resultat : (array([0.4       , 0.33333333]), array([0.125     , 0.08333333]))
#---------------------------------------------------------------------
# Calcul :
# La probabilité de jouer si temps = pluvieux 
# P(temps = pluvieux | jouer=oui) = (nbr(temps=pluvieux et jouer=oui)+alpha)/(nbr(jour=oui) + alpha * nbr_diff(temps)))
# P(temps = pluvieux | jouer=oui) = (3 + 0)/(9 + 0) ==> 3 est le nombre de différentes valeurs de temps (entrainnement)
# P(temps = pluvieux | jouer=oui) = 4/12 ==> 0.33333333333333333333333333333333333~

# La probabilité de jouer si temps = neigeux 
# P(temps = neigeux | jouer=oui) = (nbr(temps=neigeux et jouer=oui)+alpha)/(nbr(jouer=oui) + alpha * nbr_diff(temps)))
# P(temps = neigeux | jouer=oui) = (0 + 1)/(9 + 3) ==> 3 est le nombre de différentes valeurs de temps (entrainnement)
# P(temps = neigeux | jouer=oui) = 1/13 ==> 0.0833333333333333333333333333333333333~
#---------------------------------------------------------------------

P_vraiss_multi(Theta_jouer[0], "pluvieux", Theta_jouer[-1]), P_vraiss_multi(Theta_jouer[0], "neigeux", Theta_jouer[-1], alpha=1.)

### I.5. Prédiction de la classe (loi multinomiale)
Revenons maintenant à notre équation de prédiction 
$$\hat{c} = \arg\max\limits_{c_k} \log P(y=c_k) + \sum\limits_{f_j \in \overrightarrow{f}} \log P(f_j|y=c_k)$$

- On doit prédire un seule échantillon $x$. 
- La fonction doit retourner un vecteur des log-probabilité des classes
- Si anter=false donc on n'utilise pas la probabilité antérieure

In [ ]:
# TODO Réaliser la fonction de prédiction des log des probabilités
def predire(x, Theta, alpha=1., anter=True): 
    return None

#=====================================================================
# TEST UNITAIRE
#=====================================================================
# Resultat : (array([-5.20912179, -4.10264337]), array([-4.17950237, -3.66081061]))
#---------------------------------------------------------------------
predire(["pluvieux", "fraiche", "normale", "oui"], Theta_jouer), predire(["pluvieux", "fraiche", "normale", "oui"], Theta_jouer, anter=False) 

### I.6. Regrouper en une classe (loi multinomiale)

**Rien à programmer ici


In [ ]:
class NBMultinom(object): 
    
    def __init__(self, alpha=1.): 
        self.alpha = alpha
        
    def entrainer(self, X, Y):
        self.Theta = entrainer_multi(X, Y)
    
    def predire(self, X, anter=True, prob=False): 
        Y_pred = []
        cls = self.Theta[-1]["cls"]
        for i in range(len(X)): 
            log_prob = predire(X[i,:], self.Theta, alpha=self.alpha, anter=anter)
            if prob:
                Y_pred.append(np.max(log_prob))
            else:
                Y_pred.append(cls[np.argmax(log_prob)])
        return Y_pred

#=====================================================================
# TEST UNITAIRE
#=====================================================================
# Resultat : ['oui', 'non']
#---------------------------------------------------------------------
notre_modele = NBMultinom()
notre_modele.entrainer(X_jouer, Y_jouer)
X_test = np.array(
    [["neigeux", "fraiche", "normale", "oui"],
     ["neigeux", "fraiche", "haute", "oui"]
    ]
)
notre_modele.predire(X_test)

## II. Application et analyse

### II.1. Probabilité antérieure 

On veut tester l'effet de la probabilité antérieure.
Pour ce faire, nous avons entraîné deux modèles
- le premier prend en considération la probabilité antérieure
- le premier ne prend pas en considération la probabilité antérieure (considère une distribution uniforme des classes)

Pour tester si les modèles ont bien appris le dataset d'entraînement, on va les tester sur ce dataset et calculer le rapport de classification.

**TODO : Analyser les résultats**
- Que remarquez-vous ?
- Est-ce que la probabilité antérieure dans ce cas est importante ?
- Comment cette probabilité affecte le résultat ?
- Quand on est sûr que l'utilisation ou non de cette probabilité donne de même résultats ?

**Réponse**
- ...
- ...
- ... 
- ...

In [ ]:
# AVEC Scikit-learn
# ===================
from sklearn.naive_bayes import CategoricalNB
from sklearn.preprocessing import OrdinalEncoder

nb_ant = CategoricalNB(alpha=1.0, fit_prior=True)
nb_sans_ant = CategoricalNB(alpha=1.0, fit_prior=False)

enc = OrdinalEncoder()
X_jouer_enc = enc.fit_transform(X_jouer)
nb_ant.fit(X_jouer_enc, Y_jouer)
nb_sans_ant.fit(X_jouer_enc, Y_jouer)

Y_notre_ant = nb_ant.predict(X_jouer_enc)
Y_notre_sans_ant = nb_sans_ant.predict(X_jouer_enc)

# AVEC notre modèle (juste pour voir comment l'utiliser)
# =======================================================
#notre_modele = NBMultinom()
#notre_modele.entrainer(X_jouer, Y_jouer)
#Y_notre_ant = notre_modele.predire(X_jouer)
#Y_notre_sans_ant = notre_modele.predire(X_jouer, anter=False) 

# Le rapport de classification
from sklearn.metrics import classification_report

print("Notre modèle avec probabilité antérieure (a priori)")
print(classification_report(Y_notre_ant, Y_jouer))


print("Notre modèle sans probabilité antérieure (a priori)")
print(classification_report(Y_notre_sans_ant, Y_jouer))


### II.2. Lissage

On veut tester l'effet de lissage de Lidstone, on entraine 3 modèles : 
- alpha = 1 (lissage de Laplace)
- alpha = 0.5
- alpha = 0 (sans lissage)



**TODO : Analyser les résultats**
- Que remarquez-vous ?
- Est-ce que le lissage affecte la performance dans ce cas ? Pourquoi ?
- Pourquoi Scikit-learn n'accepte pas la valeur alpha=0 et affiche une alerte "UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10" ?
- Quelle est l'intéret du lissage ?

**Réponse**
- ...
- ...
- ...
- ...

In [ ]:
NBC_10 = CategoricalNB(alpha=1.0)
NBC_10.fit(X_jouer_enc, Y_jouer)

NBC_05 = CategoricalNB(alpha=0.5)
NBC_05.fit(X_jouer_enc, Y_jouer)

NBC_00 = CategoricalNB(alpha=0.)
NBC_00.fit(X_jouer_enc, Y_jouer)

Y_10 = NBC_10.predict(X_jouer_enc)
Y_05 = NBC_05.predict(X_jouer_enc)
Y_00 = NBC_00.predict(X_jouer_enc)


print("Alpha = 1.0")
print(classification_report(Y_10, Y_jouer))

print("Alpha = 0.5")
print(classification_report(Y_05, Y_jouer))

print("Alpha = 0.0")
print(classification_report(Y_00, Y_jouer))


### II.3. Comparaison avec d'autres algorithmes

Ici, on va essayer d'appliquer l'apprentissage automatique sur la détection de spam. 
Chaque message dans le dataset est représenté en utilisant un modèle "Sac à mots" (BoW : Bag of Words).
Dans l'entrainement, on récupère les différents mots qui s'apparaissent dans les messages. 
Chaque mot va être considéré comme une caractéristique. 
Donc, pour chaque message, la valeur de la caractéristique est la fréquence de son mot dans le message. 
Par exemple, si le mot "good" apparait 3 fois dans le message, donc la caractéristique "good" aura la valeur 3 dans ce message.

Notre implémentation n'est pas adéquate pour la nature de ce problème. 
Dans Scikit-learn, le [sklearn.naive_bayes.CategoricalNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.CategoricalNB.html) est similaire à notre implémentation. 
L'algorithme adéquat pour ce type de problème est [sklearn.naive_bayes.MultinomialNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html).

Le dataset utilisé est [SMS Spam Collection Dataset](https://www.kaggle.com/uciml/sms-spam-collection-dataset).
Les algorithmes comparés :
- Naive Bayes
- Arbre de décision
- Regression logistique 

In [ ]:
messages = pd.read_csv("datasets/spam.csv", encoding="latin-1")
messages = messages.rename(columns={"v1": "classe", "v2": "texte"})
messages = messages.filter(["texte", "classe"])

messages.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
#from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import timeit
from sklearn.metrics import precision_score, recall_score


temps_train = {}
temps_test = {}

rappel = {}
precision = {}

msg_train, msg_test, Y_train, Y_test = train_test_split(messages["texte"],messages["classe"],test_size=0.2, random_state=0)

count_vectorizer = CountVectorizer()
X_train = count_vectorizer.fit_transform(msg_train)
X_test = count_vectorizer.transform(msg_test)


# ==================================
# ENTRAINEMENT 
# ==================================
    
#entrainement Naive Bayes
naive_bayes = MultinomialNB()
temps_debut = timeit.default_timer()
naive_bayes.fit(X_train, Y_train)
temps_train["naive_bayes"] = timeit.default_timer() - temps_debut
    
#entrainement CART
arbre_decision = DecisionTreeClassifier()
temps_debut = timeit.default_timer()
arbre_decision.fit(X_train, Y_train)
temps_train["arbre_decision"] = timeit.default_timer() - temps_debut
    
#entrainement Régression logitique
reg_log = LogisticRegression(solver="lbfgs") #solver=sag est plus lent; donc j'ai choisi le plus rapide
temps_debut = timeit.default_timer()
reg_log.fit(X_train, Y_train)
temps_train["reg_log"] = timeit.default_timer() - temps_debut
    
# ==================================
# TEST 
# ==================================
    
#test Naive Bayes
temps_debut = timeit.default_timer()
Y_naive_bayes = naive_bayes.predict(X_test)
temps_test["naive_bayes"] = timeit.default_timer() - temps_debut
    
    
#test CART
temps_debut = timeit.default_timer()
Y_arbre_decision = arbre_decision.predict(X_test)
temps_test["arbre_decision"] = timeit.default_timer() - temps_debut
    
#test Régression logitique
temps_debut = timeit.default_timer()
Y_reg_log = reg_log.predict(X_test)
temps_test["reg_log"] = timeit.default_timer() - temps_debut
    
# ==================================
# PERFORMANCE 
# ==================================
# Ici, on va considérer une classification binaire avec une seule classe "spam" 
# On ne juge pas le classifieur sur sa capacité de détecter les non spams
    
precision["naive_bayes"] = precision_score(Y_test, Y_naive_bayes, pos_label="spam")
precision["arbre_decision"] = precision_score(Y_test, Y_arbre_decision, pos_label="spam")
precision["reg_log"] = precision_score(Y_test, Y_reg_log, pos_label="spam")
    
rappel["naive_bayes"] = recall_score(Y_test, Y_naive_bayes, pos_label="spam")
rappel["arbre_decision"] = recall_score(Y_test, Y_arbre_decision, pos_label="spam")
rappel["reg_log"] = recall_score(Y_test, Y_reg_log, pos_label="spam")
    

#### II.3.1. Temps d'entraînement et de test

Combien de temps chaque algorithme prend pour entrainer le même dataset d'entrainement et combien de temps pour tester le même dataset de test.

**TODO : Analyser les résultats**
- Que remarquez-vous concernant le temps d'entrainement ? Pourquoi nous avons eu ces résultats en se basant sur les algorithmes ?
- Que remarquez-vous concernant le temps de test ? Pourquoi nous avons eu ces résultats en se basant sur les algorithmes ?

**Réponse**
- ...
- ...

In [ ]:
pd.DataFrame({
    "Alogorithme" : ["Naive Bayes", "Arbre de decision", "Regression logistique"],
    "Temps d'entrainement" : [temps_train["naive_bayes"], temps_train["arbre_decision"], temps_train["reg_log"]],
    "Temps de test" : [temps_test["naive_bayes"], temps_test["arbre_decision"], temps_test["reg_log"]]
})

#### II.3.2. Qualité de prédiction

Comment chaque algorithme performe sur le dataset de test dans le cas de détection de spams (spam: est la classe positive).

**TODO : Analyser les résultats**

On remarque que Naive Bayes surpasse les deux autres algorithmes dans la détection de spams. (répond avec oui ou non, sans argumentation)
- Est-ce que ceci preuve que Naive Bayes est meilleur que les autres algorithmes sur n'importe quel problème ?
- Est-ce que ceci preuve que Naive Bayes peut donner de meilleurs résultats que les autres algorithmes sur des problèmes similaires ?

**Réponse**

Est-ce que ceci preuve que Naive Bayes est meilleur que les autres algorithmes sur n'importe quel problème ?
- [ ] Oui
- [ ] Non

Est-ce que ceci preuve que Naive Bayes peut donner de meilleurs résultats que les autres algorithmes sur des problèmes similaires ?
- [ ] Oui
- [ ] Non

In [ ]:
pd.DataFrame({
    "Alogorithme" : ["Naive Bayes", "Arbre de decision", "Regression logistique"],
    "Rappel" : [rappel["naive_bayes"], rappel["arbre_decision"], rappel["reg_log"]],
    "Precision" : [precision["naive_bayes"], precision["arbre_decision"], precision["reg_log"]]
})